# Immigration Data Modeling and Implementation
### Data Engineering Capstone Project
#### Project Summary
The project's goal is to build out an data pipeline that uses I94 immigratin data, city's population, temperature and airport data to create a data warehouse optimized for analyzing immigration events. The data warehouse can provide insight into the travellers' statistical information, like travel purpose and visa type, and factors affecting travellers' distribution, like number of airports in a city and average temperature of the month.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import os
import glob 
import re
import string
import configparser
import psycopg2
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, col, length, year, month, max, min, mean, asc, desc, explode, row_number, when
from pyspark.sql.types import StructType as Struct, StructField as Field, DoubleType as Double, DecimalType as Decimal, StringType as String, IntegerType as Int, LongType as Long, DateType as Date, TimestampType as Timestamp

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')
IMMIGRATION_FILE = config['FILE']['IMMIGRATION_FILE']
IMMIGRATION_LABEL_FILE = config['FILE']['IMMIGRATION_LABEL_FILE']
TEMPERATURE_FILE = config['FILE']['TEMPERATURE_FILE']
AIRPORT_FILE = config['FILE']['AIRPORT_FILE']
DEMOGRAPHICS_FILE = config['FILE']['DEMOGRAPHICS_FILE']
CHUNK_SIZE = config['PREPROCESSING']['CHUNK_SIZE']
AWS_KEY_ID = config['IAM']['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY = config['IAM']['AWS_SECRET_ACCESS_KEY']
CLUSTER_ENDPOINT = config['REDSHIFT']['HOST']
DATABASE_NAME = config['REDSHIFT']['DB_NAME']
USER_NAME = config['REDSHIFT']['DB_USER']
PASSWORD = config['REDSHIFT']['DB_PASSWORD']
CLUSTER_PORT = config['REDSHIFT']['DB_PORT']

In [3]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
                    .enableHiveSupport()\
                    .getOrCreate()

### Step 1: Scope the Project and Gather Data

#### 1.1 Scope 
The target of this project is to create a data warehouse for analyzing international travellers's statistical distribution and factors that affect the distribution. Therefore, serveral dimension tables and one fact table will be created in a AWS Redshift cluster. There are four datasets, I94 Immigration data, US Demographic Data, Weather Data and Airport data. After investigation on the, a data model will be created according to the target. Then, a data pipeline will implement the ETL, including data extracing, cleaning, transforming and loading. In the end, a data quality check will be excuted to exmaine the ETL result. 

#### 1.2 Describe and Gather Data 
#### 1.2.1 Immigration Data
This data comes from the US National Tourism and Trade Office. The link is https://travel.trade.gov/research/reports/i94/historical/2016.html. It is provided in SAS7BDAT format which is a binary database storage format. The data dictionary of immigration data is as follows,

| Column Name | Description |
| :--- | :--- |
| CICID | Primary Key |
| I94YR | Year |
| I94MON | Month |
| I94CIT | Country of Citizenship |
| I94RES | Country of Residency |
| I94PORT | Port of Arrival |
| ARRDATE | Arrival Date |
| I94MODE | Transportation Mode |
| I94ADDR | State of Arrival |
| DEPDATE | Departure Date |
| I94BIR | Age|
| I94VISA | Visa Category |
| COUNT | Number of People |
| DTADFILE | Character Date |
| VISAPOST | Department Issuing Visa |
| OCCUP | Occupation |
| ENTDEPA | Arrival Flag |
| ENTDEPD | Departure Flag |
| ENTDEPU | Update Flag |
| MATFLAG | Match Flag |
| BIRYEAR | Year of Birth |
| DTADDTO | Character Date |
| GENDER | Gender |
| INSNUM | INS number |
| AIRLINE | Airline |
| ADMNUM | Admission number |
| FLTNO | Flight Number |
| VISATYPE | Visa Type |

In [4]:
file_list = []
for root, dirs, files in os.walk(IMMIGRATION_FILE):
    for file in files:
        if file.endswith(".sas7bdat"):
            file_list.append(os.path.join(root, file))      
file = file_list[0]

In [5]:
immigration_spark_df = spark.read.format('com.github.saurfang.sas.spark').load(file)
immigration_spark_df.printSchema()
immigration_spark_df.show(5)
immigration_spark_df.count()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

3096313

#### 1.2.2 Weather Data
This data comes from Kaggle. The link is https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data. It is provided in CSV format and its data dictionary is as follow,

| Column Name | Description |
| :--- | :--- |
| dt | Date |
| AverageTemperature | Monthly Average Temperature |
| AverageTemperatureUncertainty | Monthly Average Temperature Uncertainty |
| City | City |
| Country | Country |
| Latitude | Latitude |
| Longitude | Longitude |

In [6]:
weather_spark_df = spark.read.format('csv').option("sep", ",").option("inferSchema", "true").option("header", "true").load(TEMPERATURE_FILE)
weather_spark_df.printSchema()
weather_spark_df.show(5)
weather_spark_df.count()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|                 dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+-------------------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01 00:00:00|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01 00:00:00|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01 00:00:00|              null|                         n

8599212

#### 1.2.3 US Demographic Data by City
This data comes from OpenSoft. The link is https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/. It is provided in CSV format and its data dictionay is as follows,

| Column Name | Description |
| :--- | :--- |
| City | City |
| State | State |
| Median Age | Median of Population Age |
| Male Population | Population of Male Residents |
| Female Population | Pouplation of Female Residents |
| Total Population | Population of All Residents |
| Number of Veterans | Population of Veterans |
| Foreign-born | Population of Foreign-born Residents |
| Average Household Size | Average of Household Size |
| State Code | Two-letter State Code |
| Race | Race |
| Count | Population of Race |

In [7]:
demographic_spark_df = spark.read.format('csv').option("sep", ";").option("inferSchema", "true").option("header", "true").load(DEMOGRAPHICS_FILE)
demographic_spark_df.printSchema()
demographic_spark_df.show(5)
demographic_spark_df.count()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+-------------

2891

#### 1.2.4 Airport Data
This data comes from datahub. The link is https://datahub.io/core/airport-codes#data. It is provided in CSV format and its data dictionary is as follows,

| Column Name | Description |
| :--- | :--- |
| ident | Airport Identifier |
| type | Type of Airport |
| name | Name of Airport |
| elevation_ft | Elevation of Airport in Feet |
| continent | Continent of Airport |
| iso_country | Country of Airport |
| iso_region | Country and State of Airport |
| municipality | City of Airport |
| gps_code | GPS Code of Airport |
| iata_code | IATA Code of Airport |
| local_code | Local Code of Airport |
| coordinates | Latidue and Longitude of Airport |

In [8]:
airport_spark_df = spark.read.format('csv').option("sep", ",").option("inferSchema", "true").option("header", "true").load(AIRPORT_FILE)
airport_spark_df.printSchema()
airport_spark_df.show(5)
airport_spark_df.count()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|   

55075

### Step 2: Explore and Assess the Data

#### 2.1 Immigration Data
Some columns in the immigration data have codes that needs to be tranlasted in to meaningful contents. The Immigration Data Lable file, from the US National Tourism and Trade Office, can be used to implement the translation. The file's link is https://travel.trade.gov/research/reports/i94/historical/2016.html. It is provided in SAS format. Also, after translation, the immigration data can be joined with other data. The script below reads the translation of five types of codes.
1. Country
2. Port of Arrival
3. Transportation Mode
4. Destination State
5. Visa Type

In [9]:
country_list = []
port_list = []
mode_list = []
address_list = []
visa_list = []

with open(IMMIGRATION_LABEL_FILE) as file:
    lines = file.readlines()
    
    country_flag = False
    port_flag = False
    mode_flag = False
    address_flag = False
    visa_flag = False  

    for i, line in enumerate(lines):        
        if 'value i94cntyl' in line:            
            country_flag = True
            continue
        elif 'value $i94prtl' in line:           
            port_flag = True
            continue
        elif 'value i94model' in line:            
            mode_flag = True
            continue
        elif 'value i94addrl' in line:            
            address_flag = True
            continue    
        elif '/* I94VISA' in line:            
            visa_flag = True
            continue
            
        if country_flag == True:
            if len(line.replace(';','').replace('\n','').replace('\'','').strip()) > 5:
                pair = list(line.split('='))
                dict = {}               
                dict['code'] = int(pair[0].replace('\'','').strip())
                dict['country'] = string.capwords(pair[1].replace(';','').replace('\n','').replace('\'','').strip())
                country_list.append(dict)            
            if ';' in line:
                country_flag = False
            continue
        elif port_flag == True:                      
            if len(line.replace(';','').replace('\n','').replace('\'','').strip()) > 5:
                pair = list(line.split('='))
                addr = pair[1].split(',')
                dict = {}
                dict['code'] = pair[0].replace('\'','').strip()                
                if len(addr) > 1:
                    dict['city'] = string.capwords(addr[0].replace(';','').replace('\n','').replace('\'','').strip())
                    dict['state'] = addr[1].replace(';','').replace('\n','').replace('\'','').strip().upper()
                elif 'Collapsed' in addr[0]:
                    dict['city'] = addr[0][addr[0].find("(")+1:addr[0].find(")")]
                    dict['state'] = 'NA'
                else:
                    dict['city'] = addr[0].replace(';','').replace('\n','').replace('\'','').strip()
                    dict['state'] = 'NA'
                port_list.append(dict)            
            if ';' in line:
                port_flag = False
            continue
        elif mode_flag == True:
            if len(line.replace(';','').replace('\n','').replace('\'','').strip()) > 5:
                pair = list(line.split('='))
                dict = {}
                dict['code'] = int(pair[0].replace('\'','').strip())
                dict['transport_mode'] = string.capwords(pair[1].replace(';','').replace('\n','').replace('\'','').strip())
                mode_list.append(dict)            
            if ';' in line:
                mode_flag = False
            continue         
        elif address_flag == True:
            if len(line.replace(';','').replace('\n','').replace('\'','').strip()) > 5:
                pair = list(line.split('='))
                dict = {}
                dict['code'] = pair[0].replace('\'','').strip()
                dict['state'] = string.capwords(pair[1].replace(';','').replace('\n','').replace('\'','').strip())
                address_list.append(dict)            
            if ';' in line:
                address_flag = False
            continue   
        elif visa_flag == True:
            if len(line.replace(';','').replace('\n','').replace('\'','').strip()) > 5:                
                pair = list(line.split('='))
                dict = {}
                dict['code'] = int(pair[0].replace('\'','').strip())
                dict['visa_cat'] = string.capwords(pair[1].replace(';','').replace('\n','').replace('\'','').strip())
                visa_list.append(dict)            
            if '*/' in line:
                visa_flag = False
            continue

##### 2.1.1 Country Code
The country codes are used in I94CIT and I94RES.

In [10]:
country_spark_df = spark.sparkContext.parallelize(country_list).map(lambda x: Row(**x)).toDF()
country_spark_df.printSchema()
country_spark_df.show(5)
country_spark_df.count()

root
 |-- code: long (nullable = true)
 |-- country: string (nullable = true)

+----+--------------------+
|code|             country|
+----+--------------------+
| 582|Mexico Air Sea, A...|
| 236|         Afghanistan|
| 101|             Albania|
| 316|             Algeria|
| 102|             Andorra|
+----+--------------------+
only showing top 5 rows



289

##### 2.1.2 Port Code
The port codes are composed of states and cities in the I94PORT column.

In [11]:
port_spark_df = spark.sparkContext.parallelize(port_list).map(lambda x: Row(**x)).toDF()
port_spark_df = port_spark_df
port_spark_df.printSchema()
port_spark_df.show(5)
port_spark_df.count()

root
 |-- city: string (nullable = true)
 |-- code: string (nullable = true)
 |-- state: string (nullable = true)

+--------------------+----+-----+
|                city|code|state|
+--------------------+----+-----+
|               Alcan| ALC|   AK|
|           Anchorage| ANC|   AK|
|Baker Aaf - Baker...| BAR|   AK|
|       Daltons Cache| DAC|   AK|
|Dew Station Pt La...| PIZ|   AK|
+--------------------+----+-----+
only showing top 5 rows



660

##### 2.1.3 Transportation Mode Code
The transportation mode codes are used in I94.

In [12]:
mode_spark_df = spark.sparkContext.parallelize(mode_list).map(lambda x: Row(**x)).toDF()
mode_spark_df.printSchema()
mode_spark_df.show(5)
mode_spark_df.count()

root
 |-- code: long (nullable = true)
 |-- transport_mode: string (nullable = true)

+----+--------------+
|code|transport_mode|
+----+--------------+
|   1|           Air|
|   2|           Sea|
|   3|          Land|
|   9|  Not Reported|
+----+--------------+



4

##### 2.1.4 Address Code
The address codes are used in I94ADDR.

In [13]:
address_spark_df = spark.sparkContext.parallelize(address_list).map(lambda x: Row(**x)).toDF()
address_spark_df.printSchema()
address_spark_df.show(5)
address_spark_df.count()

root
 |-- code: string (nullable = true)
 |-- state: string (nullable = true)

+----+----------+
|code|     state|
+----+----------+
|  AL|   Alabama|
|  AK|    Alaska|
|  AZ|   Arizona|
|  AR|  Arkansas|
|  CA|California|
+----+----------+
only showing top 5 rows



55

##### 2.1.5 Visa Code
The visa codes are used in I94VISA.

In [14]:
visa_spark_df = spark.sparkContext.parallelize(visa_list).map(lambda x: Row(**x)).toDF()
visa_spark_df.printSchema()
visa_spark_df.show(5)
visa_spark_df.count()

root
 |-- code: long (nullable = true)
 |-- visa_cat: string (nullable = true)

+----+--------+
|code|visa_cat|
+----+--------+
|   1|Business|
|   2|Pleasure|
|   3| Student|
+----+--------+



3

##### 2.1.6 The cleaning of immigration data includes three steps:
1. The data has many unrelated data, so the first step is to remove them. 
2. The next step is translating the types of code into readable contents using the data extract from lable description file. 
3. Since this project focuses on the statistics of immigration data related to airports, only the records whose trasportation mode are air is kept. 

In [15]:
cleaned_immigration_df = immigration_spark_df.select(col('cicid').cast(Int()),col('i94yr').cast(Int()).alias('year'), col('i94mon').cast(Int()).alias('month'), col('i94cit').alias('citizenship'),\
                                                     col('i94res').alias('residency'), col('i94port').alias('port'), col('i94mode').alias('transportation_mode'), col('i94addr').alias('arrival_state'),\
                                                     col('i94bir').cast(Int()).alias('age'), col('i94visa').alias('visa_category'),col('visatype').alias('visa_type'), col('biryear').cast(Int()).alias('birth_year'))\
                                             .join(country_spark_df, col('citizenship')==country_spark_df.code, how='left').withColumn('citizenship',col('country')).drop('code','country')\
                                             .join(country_spark_df, col('residency')==country_spark_df.code, how='left').withColumn('residency',col('country')).drop('code','country')\
                                             .join(port_spark_df, col('port')==port_spark_df.code, how='left').withColumn('port_city',col('city')).withColumn('port_state',col('state')).drop('code','city','state')\
                                             .join(mode_spark_df, col('transportation_mode')==mode_spark_df.code, how='left').withColumn('transportation_mode',col('transport_mode')).drop('code','transport_mode')\
                                             .join(address_spark_df, col('arrival_state')==address_spark_df.code, how='left').withColumn('arrival_state',col('state')).drop('code','state')\
                                             .join(visa_spark_df, col('visa_category')==visa_spark_df.code, how='left').withColumn('visa_category',col('visa_cat')).drop('code','visa_cat')\
                                             .withColumn('port_state', when(col('port_city')=='WASHINGTON DC', 'DC').otherwise(col('port_state')))\
                                             .withColumn('port_city', when(col('port_city')=='WASHINGTON DC', 'Washington').otherwise(col('port_city')))\
                                             .filter(col('transportation_mode')=='Air').orderBy(asc('cicid'))

In [ ]:
cleaned_immigration_df.printSchema()
cleaned_immigration_df.show(5)
cleaned_immigration_df.count()

root
 |-- cicid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- residency: string (nullable = true)
 |-- port: string (nullable = true)
 |-- transportation_mode: string (nullable = true)
 |-- arrival_state: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- visa_category: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- port_city: string (nullable = true)
 |-- port_state: string (nullable = true)



#### 2.2 Weather Data
The data has the monthly average temperature of cities each year from 1850 to 2013. But the date of interest is from 01/01/2016 to 12/31/2016, so I calculated the monthly average temperature of cities over years. Plus, I'm interested in the cities in United States, so only records of cities in United States are kept.

In [ ]:
cleaned_weather_df = weather_spark_df.filter(col('Country') == 'United States')\
                    .select(month('dt').alias('month'), col('AverageTemperature').alias('temperature'), col('City').alias('city'), col('Country').alias('country'))\
                    .groupBy('country', 'city', 'month').agg(mean('temperature').cast(Decimal(5,2)).alias('average_temperature')).orderBy(asc('city'), asc('month'))

In [ ]:
cleaned_weather_df.printSchema()
cleaned_weather_df.show(20)
cleaned_weather_df.count()

#### 2.3 US Demographic Data by City
The US demographic data is pretty clean. But it has one issue that there are duplicates due to muliple races in one city, so the only cleaning step is to remove the duplicate cities.

In [ ]:
cleaned_demographic_df = demographic_spark_df.select(col('State Code').alias('state_code'), col('State').alias('state'), col('City').alias('city'), col('Median Age').alias('median_age'),\
                                                     col('Male Population').alias('male_population'), col('Female Population').alias('female_population'), col('Total Population').alias('total_population'),\
                                                     col('Number of Veterans').alias('veteran_population'), col('Average Household Size').alias('average_household_size')).dropDuplicates().orderBy(asc('state_code'),asc('city'))

In [ ]:
cleaned_demographic_df.printSchema()
cleaned_demographic_df.show(5)
cleaned_demographic_df.count()

#### 2.4 Airport Data
The airport data includes airport information around the world. But the conuntry of interest is United States, so airports not in United States are removed. Second, the state name is part of iso_region, in order to get the state where the airports are located in, I applied a function to_state to the iso_region column. Besides, the latitude and longitude of airports are stored in the coordinates column, I used two functions to_latitude and to_longitude to split the coordinates into latitude and longitude. Third, some airports have no city value that is used for immigration data to find correpsonding airports, therefore, only airports whose city column is not null are kept. Last, heliports and Air Force bases are includes in the airport data, which have nothing to do with immigration and thus are removed.

In [ ]:
to_latitude = udf(lambda x: float(x.split(',')[0]), Double())
to_longitude = udf(lambda x: float(x.split(',')[1]), Double())
to_state = udf(lambda x: x.split('-')[1], String())
cleaned_airport_df = airport_spark_df.filter((col('iso_country') == 'US') & (col('municipality').isNotNull()) & (col('type').like('%airport%')) & (~col('name').like('%base%')) )\
                                     .withColumn('latitude', to_latitude('coordinates').cast(Decimal(9,6))).withColumn('longitude', to_longitude('coordinates').cast(Decimal(9,6))).withColumn('state', to_state('iso_region'))\
                                     .select(col('ident').alias('id'), col('type'), col('name'), col('elevation_ft'), col('iso_country').alias('country'), col('state'),\
                                             col('municipality').alias('city'), col('gps_code'), col('local_code'), col('latitude'),col('longitude'))\
                                     .orderBy(asc('state'), asc('city'))


In [ ]:
cleaned_airport_df.printSchema()
cleaned_airport_df.show(20)
cleaned_airport_df.count()

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
1. Project Scope:
This project is used to analysis the immigration distribution over city, temperature, traveller's citizenship, residency, visa category and visa type. Therefore, the immigration data will be used in the fact table factImmigration, and other descriptive information will be used in dimension tables, like dimCity and dimCountry.
2. Granularity:
Since the immigration data is provided in different month across cities, the granularity of fact and dimension tables is per month and per city.
3. Fact Table:
The factImmigration has the traveller's information, such as port, citizenship, residency, destination, visa category, visa type and arrvial date. 
4. Dimension Table:
There are seven dimension tables, dimCountry, dimState, dimCity, dimTemperature, dimAirport, dimVisaCategory and dimVisaType. Dimension tables will be used to dissect the factImmigration to provide statistical analysis for the immigration data.
5. Schema:
The snowflake schema is adopted. For example, dimState is used to identify the cities in dimCity. The snowflake schema has some nomorlized dimensions so that it can reduce the data redundancy. But normlized data means joins will be used in some scenarions and thus slow the query execution. It's a trade off between storage efficiency and query performance.
6. ER Diagram:
The ER diagram of the date model is as follows,
<img src="src/ER Diagram.png">

#### 3.2 Data Mapping
The data mapping between files and tables will be explained for each table.
##### 3.2.1 dimState
The state information comes from the us-cities-demographics.csv. The id column is auto incremental. The mapping between the file and dimState is as follows,

| us-cities-demographics | dimState |
| :--- | :--- |
| State | name |
| State Code | state_code |

##### 3.2.2 dimCity
The city information comes from the us-cities-demographics.csv and dimState. The id column is auto incremental. The mapping between the file and dimCity is as follows,

| us-cities-demographics | dimState | dimCity |
| :--- | :--- | :--- |
|  | id | state_id |
| City | | name |
| Median Age | | median_age |
| Male Population | | male_population |
| Female Population | | female_population |
| Total Population | | total_population |
| Number of Veterans | | veteran_population |
| Average Household Size | | average_household_size |

##### 3.2.3 dimAirport
The airport information comes from the airport-codes_csv.csv and dimCity. The id column is auto incremental. The mapping between the file and dimAirport is as follows,

| airport-codes_csv | dimCity | dimAirport |
| :--- | :--- | :--- |
|  | id | city_id |
| name | | name |
| elevation_ft | | elevation |
| type | | type |
| ident | | id_code |
| gps_code | | gps_code |
| local_code | | local_code |
| coordinates | | latitude |
| coordinates | | longitude |

##### 3.2.4 dimTemperature
The temperature information of each city comes from the GlobalLandTemperaturesByCity.csv and dimCity. The id column is auto incremental. The mapping between the file and dimTemperature is as follows,

| GlobalLandTemperaturesByCity | dimCity | dimAirport |
| :--- | :--- | :--- |
|  | id | city_id |
| dt | | month |
| AverageTemperature | | temperature |

##### 3.2.5 dimCountry
The country information comes from the immigration .sas7bdat files and I94_SAS_Labels_Descriptions.SAS. The id column is auto incremental. The country name is identified by 94_SAS_Labels_Descriptions.SAS tranlateing I94CIT/I94RES in the .sas7bdat files,

| \*.sas7bdat | I94_SAS_Labels_Descriptions | dimCountry |
| :--- | :--- | :--- |
| I94CIT/I94RES | translation | name |

##### 3.2.6 dimVisaCategory
The visa category information comes from the immigration .sas7bdat files and I94_SAS_Labels_Descriptions.SAS. The id column is auto incremental. The visa category is identified by 94_SAS_Labels_Descriptions.SAS tranlateing I94VISA in the .sas7bdat files,

| \*.sas7bdat | I94_SAS_Labels_Descriptions | dimVisaCategory |
| :--- | :--- | :--- |
| I94VISA | translation | visa_category |

##### 3.2.7 dimVisaType
The visa type information comes from the immigration .sas7bdat files and I94_SAS_Labels_Descriptions.SAS. The id column is auto incremental. The visa type is identified by 94_SAS_Labels_Descriptions.SAS tranlateing VISATYPE in the .sas7bdat files,

| \*.sas7bdat | I94_SAS_Labels_Descriptions | dimVisaType |
| :--- | :--- | :--- |
| VISATYPE | translation | visa_type |

##### 3.2.8 factImmigration
The immigration information comes from the immigration .sas7bdat files and dimension tables. The id column is auto incremental. The mapping between the file and factImmigration is as follows,

| \*.sas7bdat | dimCountry | dimState | dimCity | dimVisaCategory | dimVisaType | factImmigration |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
|  |  |  | id |  |  | city_id |
|  | id |  |  |  |  | citizenship |
|  | id |  |  |  |  | residency |
|  |  | id |  |  |  | destination |
| CICID |  |  |  |  |  | cicid |
|  |  |  |  | id |  | visa_cat_id |
|  |  |  |  |  | id | visa_type_id |
| I94YR |  |  |  |  |  | year |
| I94MON |  |  |  |  |  | month |
| BIRYEAR |  |  |  |  |  | birth_year |
| I94BIR |  |  |  |  |  | age |

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model


#### 4.1 Create Tables
The fact and dimension tables from the ER diagram are created on AWS Redshift cluster. The integrity contraints, such as primary key and foreign key, are applied to each table. After the tables are created, the queries for counting the number of rows are executed to check if the table are created successfully. 

In [23]:
conn = psycopg2.connect(f"host={CLUSTER_ENDPOINT} dbname={DATABASE_NAME} user={USER_NAME} password={PASSWORD} port={CLUSTER_PORT}")
cur = conn.cursor()

dimState_drop = ("DROP TABLE IF EXISTS dimState")
dimCity_drop = ("DROP TABLE IF EXISTS dimCity")
dimTemperature_drop = ("DROP TABLE IF EXISTS dimTemperature")
dimAirport_drop = ("DROP TABLE IF EXISTS dimAirport")
dimCountry_drop = ("DROP TABLE IF EXISTS dimCountry")
dimVisaCategory_drop = ("DROP TABLE IF EXISTS dimVisaCategory")
dimVisaType_drop = ("DROP TABLE IF EXISTS dimVisaType")
factImmigration_drop = ("DROP TABLE IF EXISTS factImmigration")

dimState_create = ("""CREATE TABLE IF NOT EXISTS dimState (
                                                          id bigint NOT NULL PRIMARY KEY,                                                          
                                                          name varchar,
                                                          state_code varchar
                                                          )""")

dimCity_create = ("""CREATE TABLE IF NOT EXISTS dimCity (
                                                          id bigint NOT NULL PRIMARY KEY,
                                                          state_id bigint,
                                                          name varchar,
                                                          median_age float,
                                                          male_population bigint,
                                                          female_population bigint,
                                                          total_population bigint,
                                                          veteran_population bigint,
                                                          average_household_size float,
                                                          FOREIGN KEY (state_id) REFERENCES dimState(id)
                                                          )""")

dimTemperature_create = ("""CREATE TABLE IF NOT EXISTS dimTemperature (
                                                          city_id bigint NOT NULL,                                                          
                                                          month int,
                                                          temperature float,
                                                          PRIMARY KEY (city_id, month),
                                                          FOREIGN KEY(city_id) REFERENCES dimCity(id)
                                                          )""")

dimAirport_create = ("""CREATE TABLE IF NOT EXISTS dimAirport (
                                                          id bigint NOT NULL PRIMARY KEY,
                                                          city_id bigint,
                                                          name varchar,
                                                          elevation int,
                                                          type varchar,
                                                          id_code varchar,
                                                          gps_code varchar,
                                                          local_code varchar,
                                                          latitude float,
                                                          longitude float,
                                                          FOREIGN KEY(city_id) REFERENCES dimCity(id)
                                                          )""")

dimCountry_create = ("""CREATE TABLE IF NOT EXISTS dimCountry (
                                                          id int NOT NULL PRIMARY KEY,                                                          
                                                          name varchar
                                                          )""")

dimVisaCategory_create = ("""CREATE TABLE IF NOT EXISTS dimVisaCategory (
                                                          id int NOT NULL PRIMARY KEY,                                                          
                                                          visa_category varchar
                                                          )""")

dimVisaType_create = ("""CREATE TABLE IF NOT EXISTS dimVisaType (
                                                          id int NOT NULL PRIMARY KEY,                                                          
                                                          visa_type varchar
                                                          )""")

factImmigration_create = ("""CREATE TABLE IF NOT EXISTS factImmigration (
                                                          id bigint  NOT NULL PRIMARY KEY,
                                                          city_id bigint,
                                                          citizenship int,
                                                          residency int,
                                                          destination int,
                                                          cicid bigint,
                                                          visa_cat_id int,
                                                          visa_type_id int,
                                                          year int,
                                                          month int,
                                                          birth_year int, 
                                                          age int,
                                                          FOREIGN KEY(city_id) REFERENCES dimCity(id),
                                                          FOREIGN KEY(citizenship) REFERENCES dimCountry(id),
                                                          FOREIGN KEY(residency) REFERENCES dimCountry(id),
                                                          FOREIGN KEY(destination) REFERENCES dimState(id),
                                                          FOREIGN KEY(visa_cat_id) REFERENCES dimVisaCategory(id),
                                                          FOREIGN KEY(visa_type_id) REFERENCES dimVisaType(id)
                                                          )""")

dimState_count = ("SELECT 'dimState' AS table_name, COUNT(1) as num_of_records FROM dimstate")
dimCity_count = ("SELECT 'dimCity' AS table_name, COUNT(1) as num_of_records FROM dimcity")
dimTemperature_count = ("SELECT 'dimTemperature' AS table_name, COUNT(1) as num_of_records FROM dimtemperature")
dimAirport_count = ("SELECT 'dimAirport' AS table_name, COUNT(1) as num_of_records FROM dimairport")
dimCountry_count = ("SELECT 'dimCountry' AS table_name, COUNT(1) as num_of_records FROM dimcountry")
dimVisaCategory_count = ("SELECT 'dimVisaCategory' AS table_name, COUNT(1) as num_of_records FROM dimvisacategory")
dimVisaType_count  = ("SELECT 'dimVisaType' AS table_name, COUNT(1) as num_of_records FROM dimvisatype")
factImmigration_count = ("SELECT 'factImmigration' AS table_name, COUNT(1) as num_of_records FROM factimmigration")

table_drop_list = [factImmigration_drop, dimVisaType_drop, dimVisaCategory_drop, dimCountry_drop, dimAirport_drop, dimTemperature_drop, dimCity_drop, dimState_drop]
table_create_list = [dimState_create, dimCity_create, dimTemperature_create, dimAirport_create, dimCountry_create, dimVisaCategory_create, dimVisaType_create, factImmigration_create]
table_count_list = [dimState_count, dimCity_count, dimTemperature_count, dimAirport_count, dimCountry_count, dimVisaCategory_count, dimVisaType_count, factImmigration_count]

for query in table_drop_list:
    cur.execute(query)
    conn.commit()

for query in table_create_list:
    cur.execute(query)
    conn.commit()
    
for query in table_count_list:
    cur.execute(query)
    result = cur.fetchone()
    print(f"The {result[0]} has {result[1]} rows.")
cur.close()
conn.close()

The dimState has 0 rows.
The dimCity has 0 rows.
The dimTemperature has 0 rows.
The dimAirport has 0 rows.
The dimCountry has 0 rows.
The dimVisaCategory has 0 rows.
The dimVisaType has 0 rows.
The factImmigration has 0 rows.


#### 4.2 Prepare the Dateframe to storage 
For each table, a pandas dataframe is created with same column names as the target table. Besides, an id column as surrogate key is added to each dataframe. The reason why I chose pandas dataframe as the staging tables is I can not install the spark-redshift package which can use spark to populate tables on the Redshift cluster. In the end, I used pandas together with SqlAlchemey to finish saving data to Redshift. 

In [24]:
#create datafrome to populate dimState
state_winwdow = Window().orderBy('name')
dimState_df = cleaned_demographic_df.select(col('state').alias('name'),'state_code').dropDuplicates().withColumn('id', row_number().over(state_winwdow))
dimState_pd_df = dimState_df.toPandas()
dimState_pd_df.head(10)

,name,state_code,id
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,3
3,Arkansas,AR,4
4,California,CA,5
5,Colorado,CO,6
6,Connecticut,CT,7
7,Delaware,DE,8
8,District of Columbia,DC,9
9,Florida,FL,10


In [25]:
#create dataframe to populate dimCity
city_window = Window().orderBy(asc('state_id'),asc('name'))
dimCity_df = cleaned_demographic_df.join(dimState_df, cleaned_demographic_df.state==dimState_df.name, how='inner')\
                                .select(col('id').alias('state_id'), col('city').alias('name'), 'median_age', 'male_population',\
                                        'female_population', 'total_population', 'veteran_population', 'average_household_size')\
                                .withColumn('id', row_number().over(city_window))
dimCity_pd_df = dimCity_df.toPandas()
dimCity_pd_df.head(10)

,state_id,name,median_age,male_population,female_population,total_population,veteran_population,average_household_size,id
0,1,Birmingham,35.6,102122.0,112789.0,214911,13212.0,2.21,1
1,1,Dothan,38.9,32172.0,35364.0,67536,6334.0,2.59,2
2,1,Hoover,38.5,38040.0,46799.0,84839,4819.0,2.58,3
3,1,Huntsville,38.1,91764.0,97350.0,189114,16637.0,2.18,4
4,1,Mobile,38.0,91275.0,103030.0,194305,11939.0,2.40,5
5,1,Montgomery,35.4,94582.0,106004.0,200586,14955.0,2.41,6
6,1,Tuscaloosa,29.1,47293.0,51045.0,98338,3647.0,2.67,7
7,2,Anchorage,32.2,152945.0,145750.0,298695,27492.0,2.77,8
8,3,Avondale,29.1,38712.0,41971.0,80683,4815.0,3.18,9
9,3,Casas Adobes,44.8,30890.0,34375.0,65265,6601.0,2.24,10


In [26]:
#create dataframe to populate dimTemperature
dimTemperature_df = cleaned_weather_df.join(dimCity_df, cleaned_weather_df.city==dimCity_df.name, how='inner')\
                                .select(col('id').alias('city_id'), 'month', col('average_temperature').alias('temperature'))\
                                .orderBy(asc('city_id'),asc('month'))                                                                       
dimTemperature_pd_df = dimTemperature_df.toPandas()
dimTemperature_pd_df.head(10)

,city_id,month,temperature
0,1,1,7.02
1,1,2,8.92
2,1,3,12.66
3,1,4,17.08
4,1,5,21.56
5,1,6,25.37
6,1,7,26.87
7,1,8,26.50
8,1,9,23.38
9,1,10,17.09


In [27]:
#create dataframe to populate dimAirport
temp_dimCity_df = dimCity_df.join(dimState_df, dimCity_df.state_id == dimState_df.id, how='inner').select(dimCity_df.id, dimCity_df.name, dimState_df.state_code)
airport_window = Window().orderBy(asc('city_id'))
dimAirport_df = cleaned_airport_df.join(temp_dimCity_df, [cleaned_airport_df.city==temp_dimCity_df.name, cleaned_airport_df.state==temp_dimCity_df.state_code], how='inner')\
                                  .select(temp_dimCity_df.id.alias('city_id'), cleaned_airport_df.name, col('elevation_ft').alias('elevation'), 'type',\
                                        cleaned_airport_df.id.alias('id_code'), 'gps_code', 'local_code', 'latitude', 'longitude')\
                                  .withColumn('id', row_number().over(airport_window))
dimAirport_pd_df = dimAirport_df.toPandas()
dimAirport_pd_df.head(10)

,city_id,name,elevation,type,id_code,gps_code,local_code,latitude,longitude,id
0,1,Birmingham-Shuttlesworth International Airport,650.0,large_airport,KBHM,KBHM,BHM,-86.753502,33.562901,1
1,1,Bonham Airport,600.0,small_airport,AL40,AL40,AL40,-86.985802,33.543201,2
2,2,Dothan Regional Airport,401.0,medium_airport,KDHN,KDHN,DHN,-85.449600,31.321301,3
3,4,Madison County Executive Airport-Tom Sharp Jr ...,756.0,small_airport,KMDQ,KMDQ,MDQ,-86.557503,34.861401,4
4,4,Huntsville International Carl T Jones Field,629.0,large_airport,KHSV,KHSV,HSV,-86.775101,34.637199,5
5,4,Big Sky Airport,830.0,small_airport,AL93,AL93,AL93,-86.709999,34.885601,6
6,4,Milton Airport,639.0,small_airport,4AL8,4AL8,4AL8,-86.673302,34.490601,7
7,4,Moontown Airport,650.0,small_airport,3M5,3M5,3M5,-86.461403,34.747299,8
8,5,Mobile Regional Airport,219.0,large_airport,KMOB,KMOB,MOB,-88.242798,30.691200,9
9,5,Mobile Downtown Airport,26.0,medium_airport,KBFM,KBFM,BFM,-88.068100,30.626801,10


In [28]:
#create dataframe to populate dimCountry
country_window = Window().orderBy(asc('country'))
dimCountry_df = country_spark_df.select(row_number().over(country_window).alias('id'),col('country').alias('name'))
dimCountry_pd_df = dimCountry_df.toPandas()
dimCountry_pd_df.head(10)

,id,name
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,Andorra
4,5,Angola
5,6,Anguilla
6,7,Antigua-barbuda
7,8,Argentina
8,9,Armenia
9,10,Aruba


In [30]:
#create dataframe to populate dimVisaCategory
dimVisaCategory_df = visa_spark_df.withColumn('id', col('code').cast(Int())).withColumnRenamed('visa_cat', 'visa_category').drop('code')
dimVisaCategory_pd_df = dimVisaCategory_df.toPandas()
dimVisaCategory_pd_df.head()

,visa_category,id
0,Business,1
1,Pleasure,2
2,Student,3


In [31]:
#create dataframe to populate dimVisaType
visa_window = Window().orderBy(asc('visa_type'))
dimVisaType_df = cleaned_immigration_df.select('visa_type').dropDuplicates().withColumn('id', row_number().over(visa_window))
dimVisaType_pd_df = dimVisaType_df.toPandas()
dimVisaType_pd_df.head(10)

,visa_type,id
0,B1,1
1,B2,2
2,CP,3
3,CPL,4
4,E1,5
5,E2,6
6,F1,7
7,F2,8
8,GMB,9
9,GMT,10


In [33]:
#create dataframe to populate factImmigration
immigration_window = Window().orderBy(asc('cicid'))
factImmigration_df = cleaned_immigration_df.join(dimCountry_df, cleaned_immigration_df.citizenship == dimCountry_df.name, how='left').withColumn('citizenship', col('id')).drop('id','name')\
                                           .join(dimCountry_df, cleaned_immigration_df.residency == dimCountry_df.name, how='left').withColumn('residency', col('id')).drop('id','name')\
                                           .join(dimState_df, cleaned_immigration_df.arrival_state == dimState_df.name, how='left').withColumn('destination', col('id')).drop('id','name','state_code')\
                                           .join(dimVisaCategory_df, cleaned_immigration_df.visa_category == dimVisaCategory_df.visa_category, how='left').withColumn('visa_cat_id', col('id'))\
                                           .drop('id','visa_category')\
                                           .join(dimVisaType_df, cleaned_immigration_df.visa_type == dimVisaType_df.visa_type, how='left').withColumn('visa_type_id', col('id')).drop('id','visa_type')\
                                           .join(temp_dimCity_df, [cleaned_immigration_df.port_city == temp_dimCity_df.name,cleaned_immigration_df.port_state == temp_dimCity_df.state_code], how='left')\
                                           .withColumn('city_id', col('id')).drop('id', 'name', 'state_code')\
                                           .select(row_number().over(immigration_window).alias('id'), 'city_id', 'citizenship', 'residency', 'destination', 'cicid', 'visa_cat_id', 'visa_type_id',\
                                                  'year', 'month', 'birth_year', 'age')                                        
#There are over 3 million rows of immigration records, it takes hours to load them into the Redshift cluster. So I took 10,000 records as an example. 
#If the whole dataset needs to be loaded, please comment out the next line.
factImmigration_df = factImmigration_df.filter(col('id')<=10000)

factImmigration_pd_df = factImmigration_df.toPandas()
factImmigration_pd_df.head(10)

,id,city_id,citizenship,residency,destination,cicid,visa_cat_id,visa_type_id,year,month,birth_year,age
0,1,244.0,NaN,246,1.0,7,3,7,2016,4,1991,25
1,2,193.0,2.0,2,23.0,15,2,2,2016,4,1961,55
2,3,411.0,2.0,2,22.0,16,2,2,2016,4,1988,28
3,4,411.0,2.0,2,22.0,17,2,2,2016,4,2012,4
4,5,411.0,2.0,2,23.0,18,1,1,2016,4,1959,57
5,6,411.0,2.0,2,31.0,19,2,2,2016,4,1953,63
6,7,411.0,2.0,2,31.0,20,2,2,2016,4,1959,57
7,8,411.0,2.0,2,33.0,21,2,2,2016,4,1970,46
8,9,411.0,2.0,2,33.0,22,1,1,2016,4,1968,48
9,10,411.0,2.0,2,33.0,23,2,2,2016,4,1964,52


#### 4.3 Populate Date into Tables
Once the dataframes are created, the to_sql function in pandas can be used to populate the dimension and fact tables on the Redshift cluster. 

In [35]:
redshift_conn = create_engine(f"postgresql://{USER_NAME}:{PASSWORD}@{CLUSTER_ENDPOINT}:{CLUSTER_PORT}/{DATABASE_NAME}")
dimState_pd_df.to_sql('dimstate', con=redshift_conn, index=False, if_exists='append')
dimCity_pd_df.to_sql('dimcity', con=redshift_conn, index=False, if_exists='append')
dimTemperature_pd_df.to_sql('dimtemperature', con=redshift_conn, index=False, if_exists='append')
dimAirport_pd_df.to_sql('dimairport', con=redshift_conn, index=False, if_exists='append')
dimCountry_pd_df.to_sql('dimcountry', con=redshift_conn, index=False, if_exists='append')
dimVisaCategory_pd_df.to_sql('dimvisacategory', con=redshift_conn, index=False, if_exists='append')
dimVisaType_pd_df.to_sql('dimvisatype', con=redshift_conn, index=False, if_exists='append')
factImmigration_pd_df.to_sql('factimmigration', con=redshift_conn, index=False, if_exists='append')
redshift_conn.dispose()

#### 4.4 Data Quality Checks
After loading dataframes into dimension and fact tables, the number of records in each table will be examined to ensure the data quality.

In [36]:
conn = psycopg2.connect(f"host={CLUSTER_ENDPOINT} dbname={DATABASE_NAME} user={USER_NAME} password={PASSWORD} port={CLUSTER_PORT}")
cur = conn.cursor()

dimState_count = ("SELECT 'dimState' AS table_name, COUNT(1) as num_of_records FROM dimstate")
dimCity_count = ("SELECT 'dimCity' AS table_name, COUNT(1) as num_of_records FROM dimcity")
dimTemperature_count = ("SELECT 'dimTemperature' AS table_name, COUNT(1) as num_of_records FROM dimtemperature")
dimAirport_count = ("SELECT 'dimAirport' AS table_name, COUNT(1) as num_of_records FROM dimairport")
dimCountry_count = ("SELECT 'dimCountry' AS table_name, COUNT(1) as num_of_records FROM dimcountry")
dimVisaCategory_count = ("SELECT 'dimVisaCategory' AS table_name, COUNT(1) as num_of_records FROM dimvisacategory")
dimVisaType_count  = ("SELECT 'dimVisaType' AS table_name, COUNT(1) as num_of_records FROM dimvisatype")
factImmigration_count = ("SELECT 'factImmigration' AS table_name, COUNT(1) as num_of_records FROM factimmigration")

table_count_list = [dimState_count, dimCity_count, dimTemperature_count, dimAirport_count, dimCountry_count, dimVisaCategory_count, dimVisaType_count, factImmigration_count]
for query in table_count_list:
    cur.execute(query)
    result = cur.fetchone()    
    print(f"The {result[0]} has {result[1]} rows.")
    
cur.close()
conn.close()

The dimState has 49 rows.
The dimCity has 596 rows.
The dimTemperature has 3132 rows.
The dimAirport has 930 rows.
The dimCountry has 289 rows.
The dimVisaCategory has 3 rows.
The dimVisaType has 17 rows.
The factImmigration has 10000 rows.


#### 4.5 Data dictionary 
The data dictionary can be used to provide auxiliary information to the dimension and fact tables.
##### dimState

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| name | name of state | us-cities-demographics.csv |
| state_code | 2-letter code of state |  us-cities-demographics.csv |

##### dimCity

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| state_id | id in the dimState | dimState |
| name | name of city | us-cities-demographics.csv |
| median_age | median age of all residents | us-cities-demographics.csv |
| male_population | population of male residents | us-cities-demographics.csv |
| female_population | population of female residents | us-cities-demographics.csv |
| toal_population | population of all residents | us-cities-demographics.csv |
| veteran_population | population of veterans | us-cities-demographics.csv |
| average_household_dize | average number of people in a household | us-cities-demographics.csv |

##### dimTemperature

| Column Name | Description | Source |
| :--- | :--- | :--- |
| city_id | id in dimCity, primary key | dimCity |
| month | the number of month, primary key | GlobalLandTemperaturesByCity.csv |
| temperature | monthly average temperature | us-cities-demographics.csv |

##### dimAirport

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| city_id | id in dimCity | dimCity |
| name | monthly average temperature | us-cities-demographics.csv |
| elevation | elevation of airport in feet | airport-codes_csv.csv |
| type | type of airport | airport-codes_csv.csv |
| id_code | global identifier of airport | airport-codes_csv.csv |
| gps_code | gps identifier of airport | airport-codes_csv.csv |
| local_code | loca identifier of airport | airport-codes_csv.csv |
| latitude | latitude of airpot | airport-codes_csv.csv |
| longitude | longitude of airport | airport-codes_csv.csv |

##### dimCountry

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| name | name of country | I94_SAS_Labels_Descriptions.SAS |

##### dimVisaCategory

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| visa_category | category of visa | I94_SAS_Labels_Descriptions.SAS |

##### dimVisaType

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| visa_type | type of visa | \*.sas7bdat |

##### factImmigration

| Column Name | Description | Source |
| :--- | :--- | :--- |
| id | primary key | auto-incremental |
| city_id | id in dimCity | dimCity |
| citizenship | id in dimCountry | dimCountry |
| residency | id in dimCountry | dimCountry |
| detination | id in dimState | dimState |
| cicid | unique identifier on I94 Immigration Data | \*.sas7bdat |
| visa_cat_id | id in dimVisaCategory | dimVisaCategory |
| visa_type_id | id in dimVisaType | dimVisaType |
| year | year of arrival | \*.sas7bdat |
| month | month of arrival | \*.sas7bdat |
| birth_year | year of birth of traveller | \*.sas7bdat |
| age | age of traveller | \*.sas7bdat |


#### Step 5: Complete Project Write Up
1. Clearly state the rationale for the choice of tools and technologies for the project. \
In this project, I used Apache Spark to extract data from files and clean data for loading to the data warehouse. Spark can use a cluster to achieve distributed processing and thus can handel a large amount of data. Besides, Spark use in-memory technology to process data so that it can be used to build high performance data pipeline. The Pandas is used to populate data into Redshift. The reason why I use Pandas is I can not install spark-redshift package that allows Spark to populate date directly to Redshift. However, Pandas uses the in-memory technology so it's still relatively fast. The Redshift cluster is used as the data warehouse due to its high performance in query and easy to set up. 
2. Propose how often the data should be updated and why.\
Since the time granularity of the dimension and fact table is month, the incremental load should happen every month.
3. Write a description of how you would approach the problem differently under the following scenarios:
 *  The data was increased by 100x.\
Since Spark is easy to scale-out, we can add more nodes to handle data of 100 times larger. In addition, the Redshift cluster also support horizontal scaling, so adding more nodes in Redshift is also a big leap in permance when data size is 100 times larger. 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.\
The job scheduler can be used to kick off the data pipeline at flexible schedule, like some time in a day or month. There are plenty of job schedulers in the market, and the most popular two tools are Apache Airflow and OOzie. For exmaple, say it takes one hour to run the data pipeline, we can set the schedule that the job is triggered 1:00 AM every day. If job failed, the job scheduler will send email to notice product support to fix the error, which will ensure the on-time data delivery. 
 * The database needed to be accessed by 100+ people.\
There are two ways to improve the database performance, vertical and horizontal scalling. Vertical scaling means increasing the power of a single database server and horizontal scaling adding more nodes to the database server pool. By comparison, there is limitation on power of the single database server and it's expensive, horizontal scaling has no limitation and cheaper cost. When using cloud computation, both vertical and horizontal scaling are easy to achieve. For example, Redshift provides many tiers of nodes and options of the number of nodes. But from my view, I will apply auto-scaling to the Redshift cluster. As a result, the cluster can adjust its computing power according to the number of requests.  